In [ ]:
from trading_engine.core import (
    read_data, create_model_state, orchestrate_model_backtests,
    orchestrate_model_simulations, orchestrate_portfolio_optimizations,
    orchestrate_portfolio_simulations, orchestrate_portfolio_aggregation
)

from trading_engine.aggregators import AGGREGATORS
from trading_engine.model_state import FEATURES
from trading_engine.models import MODELS
from trading_engine.optimizers import OPTIMIZERS

import datetime
from typing import Callable

import polars as pl
from polars import LazyFrame
import matplotlib.pyplot as plt

In [ ]:
def ExampleModel() -> Callable[[LazyFrame], LazyFrame]:
    """
    Example model that buys and holds SPY
    """

    def run_model(lf: LazyFrame) -> LazyFrame:
        sig = (
            lf.filter(pl.col("ticker") == "SPY-US")
            .select([
                pl.col("date"),
            ])
        )
        
        weights = sig.select([
            pl.col("date"),
            pl.lit(1.0)
            .alias("SPY-US")
        ])

        return weights

    return run_model

In [ ]:
# 1) Config
# a) Config that is common across both backtests:
#      universe, date range, aggregator, optimizer, initial capital
universe = [
  "SPY-US", "SLV-US", "GLD-US", "TLT-US", "USO-US", "UNG-US", "IXJ-US",
  "KXI-US", "JXI-US", "IXG-US", "IXN-US", "RXI-US", "MXI-US", "EXI-US",
  "IXC-US", "IEI-US", "SHY-US", "BIL-US", "JPXN-US", "INDA-US", "MCHI-US",
  "EZU-US", "IBIT-US", "ETHA-US", "VIXY-US"
]
start_date = datetime.date(2024, 1, 1)
end_date = datetime.date(2025, 1, 1)
aggregators = ["model_mvo"]
optimizers = ["mean_variance_constrained"]
initial_value = 1_000_000
# b) Config that is different: model registry.
# For the reduced backtest, we will just use MODELS as the registry
# For the full backtests, we will copy MODELS and add the research model to it.
full_backtest_registry = MODELS.copy()
# Add your research model to the portfolio
full_backtest_registry["example_model"] = {
    "tickers": ["SPY-US"],  # define the tickers your model looks at
    "columns": [],  # define the model state columns your model needs
    "function": ExampleModel(),
}
# This list of models comes from https://github.com/Hawk-Center/trading-engine/blob/main/src/production/paper/config.yaml
# Make sure to check to ensure this list is still up to date.
reduced_backtest_models = ['TLT_AMMA', 'IEI_AMMA', 'SHY_AMMA', 'BIL_AMMA', 'SLV_AMMA', 'GLD_AMMA', 'SPY_AMMA', 'INDA_AMMA', 'MCHI_AMMA', 'IBIT_AMMA', 'ETHA_AMMA', 'EZU_AMMA']
full_backtest_models = reduced_backtest_models + ["example_model"]

In [ ]:
# 2) Model State
raw_data_bundle = read_data(include_supplemental=True)
model_state_bundle, prices = create_model_state(
    raw_data_bundle=raw_data_bundle,
    features=list(FEATURES.keys()),  # Pass feature names, not the registry
    start_date=start_date,
    end_date=end_date,
    universe=universe
)

In [ ]:
# 3) Create a function to run a backtest
def run_backtest(models_registry, models):
    model_insights = orchestrate_model_backtests(
        model_state_bundle=model_state_bundle,
        models=models,
        universe=universe,
        registry=models_registry
    )
    
    model_simulations = orchestrate_model_simulations(
        prices=prices,
        model_insights=model_insights,
        start_date=start_date,
        end_date=end_date
    )
    aggregated_insights = orchestrate_portfolio_aggregation(
        model_insights=model_insights,
        backtest_results=model_simulations,
        universe=universe,
        aggregators=aggregators,
        start_date=start_date,
        end_date=end_date,
    )
    
    optimizer_insights = orchestrate_portfolio_optimizations(
        prices=prices,
        aggregated_insights=aggregated_insights,
        universe=universe,
        optimizers=optimizers,
    )
    
    optimizer_simulations = orchestrate_portfolio_simulations(
        prices=prices,
        portfolio_insights=optimizer_insights,
        start_date=start_date,
        end_date=end_date,
        initial_value=initial_value,
    )
    return optimizer_simulations

In [ ]:
# 4) Run FULL backtest and MARGINAL backtest
full_backtest_results = run_backtest(full_backtest_registry, full_backtest_models)
reduced_backtest_results = run_backtest(MODELS, reduced_backtest_models)

In [ ]:
# 5) Extract the returns
full_backtest_returns = full_backtest_results["mean_variance_constrained"]["backtest_results"]
reduced_backtest_returns = reduced_backtest_results["mean_variance_constrained"]["backtest_results"]

In [ ]:
# 5) Construct the marginal return stream
base = full_backtest_returns.select([
        "date",
        pl.col("daily_return").alias("daily_return_base")
    ])

reduced = reduced_backtest_returns.select([
    "date",
    pl.col("daily_return").alias("daily_return_reduced")
])

# Join on date
merged = base.join(reduced, on="date", how="inner")
marginal_returns = merged.with_columns(
    (pl.col("daily_return_base") - pl.col("daily_return_reduced"))
    .alias("daily_return")
).select(["date", "daily_return"])

In [ ]:
# 6) Plot marginal equity curve
marginal_df = marginal_returns.with_columns(
    pl.col("date").str.to_date(),
    (
        (1 + pl.col("daily_return")).cum_prod()
    ).alias("cumulative_return"),
)
full_df = full_backtest_returns.with_columns(
    pl.col("date").str.to_date(),
    (
        (1 + pl.col("daily_return")).cum_prod()
    ).alias("cumulative_return"),
)

# Plot directly from lists
plt.figure(figsize=(12, 6))
plt.plot(marginal_df['date'], marginal_df['cumulative_return'], label="Marginal Equity Curve")
plt.plot(full_df['date'], full_df['cumulative_return'], label="Full Portfolio Equity Curve")
plt.title('Equity Curve')
plt.xlabel('Date')
plt.ylabel('Equity')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()